# Data Augmentation 

In [27]:
from keras.preprocessing.image import ImageDataGenerator

# Create image augmenter
dataAugment = ImageDataGenerator(rescale=1.0/255,
                                 rotation_range=40,
                                 width_shift_range=0.2,
                                 height_shift_range=0.2,
                                 zoom_range=0.2,
                                 shear_range=0.2,
                                 fill_mode='reflect',
                                 horizontal_flip=True,
                                 brightness_range=
                                 validation_split=0.2,
                                 data_format='channels_last'
                                )
        
trainDataGenerator = dataAugment.flow_from_directory(directory='Data/Train',
                                                     target_size=(224, 224),
                                                     class_mode='categorical',
                                                     subset='training',
                                                     shuffle=True,
                                                     # seed=
                                                     # batch_size=
                                                    )

validationDataGenerator = dataAugment.flow_from_directory(directory='Data/Train',
                                                          target_size=(224,224),
                                                          class_mode='categorical',
                                                          subset='validation',
                                                          shuffle=True,
                                                          # seed=
                                                          # batch_size=
                                                         )

Found 60 images belonging to 5 classes.
Found 14 images belonging to 5 classes.


# VGG16 Model

In [32]:
from keras.applications.vgg16 import VGG16

vgg16 = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

In [37]:
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras import optimizers, models

# Freeze the layers except the last 4 layers
for layer in vgg16.layers: #[:-4]:
    layer.trainable = False

# Check the trainable status of the individual layers
for layer in vgg16.layers:
    print(layer, layer.trainable)

model = models.Sequential()
model.add(vgg16)

model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(5, activation='softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer=optimizers.RMSprop(lr=1e-3),
              metrics=['accuracy'])

<keras.engine.input_layer.InputLayer object at 0xb7d0a5f60> False
<keras.layers.convolutional.Conv2D object at 0xb4fac9748> False
<keras.layers.convolutional.Conv2D object at 0xb793aff60> False
<keras.layers.pooling.MaxPooling2D object at 0xb8e28e860> False
<keras.layers.convolutional.Conv2D object at 0xb8e28e198> False
<keras.layers.convolutional.Conv2D object at 0xb8e2922b0> False
<keras.layers.pooling.MaxPooling2D object at 0xb8e295470> False
<keras.layers.convolutional.Conv2D object at 0xb8e295be0> False
<keras.layers.convolutional.Conv2D object at 0xb8e294048> False
<keras.layers.convolutional.Conv2D object at 0xb8e2990f0> False
<keras.layers.pooling.MaxPooling2D object at 0xb8e29d630> False
<keras.layers.convolutional.Conv2D object at 0xb8e29dda0> False
<keras.layers.convolutional.Conv2D object at 0x10ea8e5f8> False
<keras.layers.convolutional.Conv2D object at 0x10ea5f710> False
<keras.layers.pooling.MaxPooling2D object at 0x10ea95160> False
<keras.layers.convolutional.Conv2D obj

In [38]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Model)                (None, 7, 7, 512)         14714688  
_________________________________________________________________
flatten_8 (Flatten)          (None, 25088)             0         
_________________________________________________________________
dense_11 (Dense)             (None, 256)               6422784   
_________________________________________________________________
dropout_6 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_12 (Dense)             (None, 5)                 1285      
Total params: 21,138,757
Trainable params: 6,424,069
Non-trainable params: 14,714,688
_________________________________________________________________


In [39]:
model.fit_generator(generator=trainDataGenerator,
                    steps_per_epoch=trainDataGenerator.samples/trainDataGenerator.batch_size,
                    epochs=10,
                    validation_data=validationDataGenerator,
                    validation_steps=validationDataGenerator.samples/validationDataGenerator.batch_size,
                    verbose=1
                   )
model.save('first_try.h5')

Epoch 1/10
1/1 [===============>..............] - ETA: 3:24 - loss: 1.9884 - acc: 0.2500

KeyboardInterrupt: 

# InceptionResNetV2 Model

In [7]:
from keras.applications.inception_resnet_v2 import InceptionResNetV2

IncResNet = InceptionResNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3), pooling='max')

In [8]:
IncResNet.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
conv2d_204 (Conv2D)             (None, 111, 111, 32) 864         input_2[0][0]                    
__________________________________________________________________________________________________
batch_normalization_204 (BatchN (None, 111, 111, 32) 96          conv2d_204[0][0]                 
__________________________________________________________________________________________________
activation_204 (Activation)     (None, 111, 111, 32) 0           batch_normalization_204[0][0]    
__________________________________________________________________________________________________
conv2d_205

__________________________________________________________________________________________________
activation_400 (Activation)     (None, 5, 5, 192)    0           batch_normalization_400[0][0]    
__________________________________________________________________________________________________
conv2d_401 (Conv2D)             (None, 5, 5, 224)    129024      activation_400[0][0]             
__________________________________________________________________________________________________
batch_normalization_401 (BatchN (None, 5, 5, 224)    672         conv2d_401[0][0]                 
__________________________________________________________________________________________________
activation_401 (Activation)     (None, 5, 5, 224)    0           batch_normalization_401[0][0]    
__________________________________________________________________________________________________
conv2d_399 (Conv2D)             (None, 5, 5, 192)    399360      block8_8_ac[0][0]                
__________

# Testing

In [4]:
i=0
for batch in dataAugment.flow_from_directory('CoinTrain',
                                             target_size=(224, 224),
                                             save_to_dir='Augmented Coins',
                                             # save_prefix='nickel'
                                             save_format='jpeg',
                                             class_mode='categorical'
                                             # seed = 
                                             # batch_size = 
                                             # subset = 
                                             # shuffle=True
                                            ):
    print(batch)
    i+=1
    if i >= 2:
        break

Found 26 images belonging to 4 classes.


In [1]:
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img

datagen = ImageDataGenerator(rescale=1.0/255.0,
                             # rotation_range=40,
                             width_shift_range=[500],
                             height_shift_range=[500],
                             # zoom_range=0.9,
                             # shear_range=0.2,
                             # horizontal_flip=True,
                             # validation_split=0.2,
                             # preprocessing_function
                             fill_mode='reflect'
                            )

Using TensorFlow backend.


In [2]:
img = load_img('CoinTrain/Quarters/Quarter5_T.jpg')  # this is a PIL image
x = img_to_array(img)  # this is a Numpy array with shape (3, 150, 150)
x = x.reshape((1,) + x.shape)  # this is a Numpy array with shape (1, 3, 150, 150)

In [4]:
# i = 0
# for batch in datagen.flow_from_directory(
#                     'CoinTrain',
#                     target_size=(224, 224),
#                     save_to_dir='Augmented Coins',
#                     save_format='jpeg'):
#     i += 1
#     if i >= 5:
#         break  # otherwise the generator would loop indefinitely
        
i = 0
for batch in datagen.flow(x, batch_size=1,
                          save_to_dir='TestFolder',
                          save_format='jpeg',
                          save_prefix='wrap'):
    i += 1
    if i >= 1:
        break

TypeError: flow() got an unexpected keyword argument 'target_size'

In [54]:
# Create image generator using augmenter
trainDataGenerator = dataAugment.flow_from_directory('CoinTrain',
                                                     target_size=(256, 256),
                                                     save_to_dir='Augmented Coins',
                                                     # save_prefix='nickel'
                                                     save_format='jpeg')

Found 26 images belonging to 4 classes.
Found 8 images belonging to 4 classes.


In [60]:
# Create network
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense

model = Sequential()
model.add(Conv2D(filters=32, kernel_size=(3,3), activation='relu', data_format='channels_last', input_shape=(256,256,3)))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(filters=32, kernel_size=(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(filters=64, kernel_size=(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(4, activation='sigmoid'))

model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])
              

In [62]:
batch_size=4

model.fit_generator(trainDataGenerator,
                    steps_per_epoch=26//batch_size,
                    epochs=2,
                    validation_data=testDataGenerator,
                    validation_steps=8//batch_size)
model.save_weights('first_try')

Epoch 1/2
6/6 [==============================] - 98s 16s/step - loss: 1.2766 - acc: 0.3782 - val_loss: 2.0103 - val_acc: 0.2500
Epoch 2/2
6/6 [==============================] - 110s 18s/step - loss: 1.2522 - acc: 0.4231 - val_loss: 1.3898 - val_acc: 0.2500
